In [17]:
import sys, os
sys.path.append(os.path.abspath("/Users/DavidHolzwarth/Uni/EPFL/bachelors-thesis"))

import numpy as np
from LP import resolve_delegations_LP, invert_graph
from graph_viz import visualize_delegation_graph

np.random.seed(1)

WEIGHTS = [(i + 1) / 10 for i in range(1, 10)]
NUM_NODES = 10
NUM_LOOPS = 5

def get_random_delegation_weights(n: int) -> list:
    """
    Generates at most `n` random weights from the set of {0.1, 0.2, ..., 0.9} such that they sum to exactly 1.0.

    Parameters:
    - n (int): The number of weights to generate.

    Returns:
    - list[float]: A list of `n` values that sum to 1.0.

    Notes:
        - If `n == 0`, an empty list is returned.
        - The algorithm may find less than 'n' weights, if e.g. n = 2, but the first weight is 1.0, then it will only return [1.0]
        - The last weight is explicitly adjusted to correct any rounding errors.
        - The algorithm ensures no negative values by rejecting choices that exceed the remaining sum.
    """
    if n == 0: return []

    diff = 1
    weights = []
    for i in range(n-1):
        for j in range(11):
            # Tries 10 times to find a valid weight. If this is not possible the algorithm gives up
            choice = np.random.choice(WEIGHTS)
            if diff - choice > 0:
                weights += [choice]
                diff -= weights[-1]
                break
    
    # If we have not used all the weights, add the last one
    if diff > 0:
        weights += [round(diff, 1)]

    return weights

def is_connected_to_sink(delegations, start_node):

    visited = set()

    def dfs(node):
        if node in visited:
            return False 
        
        visited.add(node)

        # A sink is a node with no outgoing edges 
        if node not in delegations or not delegations[node]:  
            return True  

        # Recur for all outgoing connections
        return any(dfs(neighbor) for neighbor in delegations[node])

    return dfs(start_node)

nodes = []
delegations = {}
for i in range(NUM_NODES):
    node = str(i)
    nodes += [node]
    # Choose how many delegations this node shall have
    num_delegations = np.random.randint(0, min(3, i) + 1)
    if num_delegations > 0:
        delegation_weights = get_random_delegation_weights(num_delegations)

        delegates = np.random.choice(nodes, num_delegations, replace=False)
        # Sorting assures that if the node delegates to itself, this delegation is first in the list.
        delegates = sorted(delegates, reverse=True) 

        for j in range(len(delegation_weights)):
            delegate = delegates[j]

            # Avoids delegations of N -1-> N, which are not allowed (since N is not voting and voting at the same time)
            if (delegate == node) and ((delegation_weights[j] == 1) or delegations.get(node, {}).get(node, 0) + delegation_weights[j] == 1):
                # If the only delegate is itself, turn this node into a sink by just not adding the delegation 
                if len(delegates) == 1:
                    break 
                # Else remove the delegation, and add the weight to the next delegate's weight
                else:
                    delegation_weights[j + 1] +=  delegation_weights[j] 
                    continue

            else:
                delegations.setdefault(node, {}).setdefault(delegate, 0)
                delegations[node][delegate] += delegation_weights[j]

for i in range(NUM_LOOPS):
    
    # Try to find a non-sink node with which to create a loop
    node = None
    for j in range(len(nodes)):
        node = np.random.choice(nodes)
        if node not in delegations or len(delegations[node]) == 0:
            node = None
        else:
            break

    for j, delegate in enumerate(delegations.get(node, {})):
        # Try to create a loop between this node and the selected delegate

        # Delegate is a sink
        if not delegations.get(delegate):
            # Add the loop
            delegations[delegate] = {node: 1}

            # Check that we did not create an illegal cycle
            if not is_connected_to_sink(delegations, node):
                # Remove the loop
                del delegations[delegate]
                continue
        # Delegate is not a sink
        else:
            delegates_delegates = delegations[delegate]

            # Choose any weight from the set of weights, except 1
            # Adding a new delegation with weight 1 might create a cycle
            new_delegation_weight = np.random.choice([weight for weight in WEIGHTS if weight not in [1]])

            # Free up weight from the other delegations
            for k, v in delegations[delegate].items():
                    delegations[delegate][k] = (1 - new_delegation_weight) * v

            # Add this weight directed toward the node, to create a loop 
            delegations[delegate].setdefault(node, 0)
            delegations[delegate][node] += new_delegation_weight

            break


%timeit resolve_delegations_LP(invert_graph(delegations), nodes)
powers, sink_nodes = resolve_delegations_LP(invert_graph(delegations), nodes)
display(powers)
for node in nodes:
    if node not in sink_nodes:
        powers[node] = 0
display(powers)
visualize_delegation_graph(delegations, powers)


OrderedDict({'Constraint_0': 1*0 + -1*1 + -1 = 0, 'Constraint_1': 1*1 + -1*6 + -1 = 0, 'Constraint_2': 1*2 + -0.05600000000000001*4 + -0.1*8 + -1.0 = 0, 'Constraint_3': 1*3 + -0.11200000000000002*4 + -0.6*7 + -0.9*8 + -1.0 = 0, 'Constraint_4': 0.16800000000000015*4 + -1.0 = 0, 'Constraint_5': 1*5 + -1 = 0, 'Constraint_6': 1*6 + -1 = 0, 'Constraint_7': 1*7 + -1.0*9 + -1.0 = 0, 'Constraint_8': 1*8 + -1 = 0, 'Constraint_9': -0.4*7 + 1*9 + -1.0 = 0, 'SinkNodesConstraint': 1*0 + 1*2 + 1*3 + 1*5 + -10 = 0})
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/thesis/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/q9/0xq_2r1503z58vb1xx__4r5m0000gn/T/f607823fdbd344ffb5ee83945a76bab9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/q9/0xq_2r1503z58vb1xx__4r5m0000gn/T/f607823fdbd344ffb5ee83945a76bab9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 C

{'0': 3.0,
 '1': 2.0,
 '2': 1.4333333,
 '3': 4.5666667,
 '4': 5.952381,
 '5': 1.0,
 '6': 1.0,
 '7': 3.3333333,
 '8': 1.0,
 '9': 2.3333333}

{'0': 3.0,
 '1': 0,
 '2': 1.4333333,
 '3': 4.5666667,
 '4': 0,
 '5': 1.0,
 '6': 0,
 '7': 0,
 '8': 0,
 '9': 0}

Parsing the data...Done


No trigger
Modifying node size using  power
No trigger
